In [15]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

original_crime_df = pd.read_csv("/Users/mayur/Desktop/ALDA Project/Crime_Data_from_2020_to_Present 2.csv")
print(original_crime_df.head())
print(original_crime_df['Crm Cd'].value_counts()[:10])
new_dataset = original_crime_df[original_crime_df["Crm Cd"].isin([510,624,740,330,310,230,626,440,420,354]) ]
new_dataset.head()

       DR_NO               Date Rptd                DATE OCC  TIME OCC  AREA  \
0   10304468  01/08/2020 12:00:00 AM  01/08/2020 12:00:00 AM      2230     3   
1  190101086  01/02/2020 12:00:00 AM  01/01/2020 12:00:00 AM       330     1   
2  201220752  09/16/2020 12:00:00 AM  09/16/2020 12:00:00 AM      1230    12   
3  191501505  01/01/2020 12:00:00 AM  01/01/2020 12:00:00 AM      1730    15   
4  191921269  01/01/2020 12:00:00 AM  01/01/2020 12:00:00 AM       415    19   

     AREA NAME  Rpt Dist No  Part 1-2  Crm Cd  \
0    Southwest          377         2     624   
1      Central          163         2     624   
2  77th Street         1259         2     745   
3  N Hollywood         1543         2     745   
4      Mission         1998         2     740   

                                         Crm Cd Desc  ... Status  Status Desc  \
0                           BATTERY - SIMPLE ASSAULT  ...     AO  Adult Other   
1                           BATTERY - SIMPLE ASSAULT  ...     

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468
10,200100509,01/04/2020 12:00:00 AM,01/04/2020 12:00:00 AM,2200,1,Central,192,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,NaN,NaN,NaN,15TH,OLIVE,34.0359,-118.2648
15,201811813,05/28/2020 12:00:00 AM,05/25/2020 12:00:00 AM,200,18,Southeast,1851,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,MENLO AV,130TH ST,33.9144,-118.2894


In [16]:
dataset_510 = original_crime_df[original_crime_df["Crm Cd"].isin([510]) ]
dataset_624 = original_crime_df[original_crime_df["Crm Cd"].isin([624])]
dataset_740 = original_crime_df[original_crime_df["Crm Cd"].isin([740])]
dataset_330 = original_crime_df[original_crime_df["Crm Cd"].isin([330])]
dataset_310 = original_crime_df[original_crime_df["Crm Cd"].isin([310])]
dataset_230 = original_crime_df[original_crime_df["Crm Cd"].isin([230])]
dataset_626 = original_crime_df[original_crime_df["Crm Cd"].isin([626])]
dataset_440 = original_crime_df[original_crime_df["Crm Cd"].isin([440])]
dataset_420 = original_crime_df[original_crime_df["Crm Cd"].isin([420])]
dataset_354 = original_crime_df[original_crime_df["Crm Cd"].isin([354])]


dataset_510


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
15,201811813,05/28/2020 12:00:00 AM,05/25/2020 12:00:00 AM,200,18,Southeast,1851,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,MENLO AV,130TH ST,33.9144,-118.2894
24,201713253,09/15/2020 12:00:00 AM,09/14/2020 12:00:00 AM,1305,17,Devonshire,1776,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,16700 LABRADOR ST,NaN,34.2493,-118.4965
27,200505212,01/29/2020 12:00:00 AM,01/26/2020 12:00:00 AM,1200,5,Harbor,529,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,E,BROAD,33.7753,-118.2456
32,200117988,09/15/2020 12:00:00 AM,09/03/2020 12:00:00 AM,2000,1,Central,111,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,500 N FIGUEROA ST,NaN,34.0615,-118.2470
50,201312607,06/21/2020 12:00:00 AM,06/21/2020 12:00:00 AM,30,13,Newton,1321,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,100 E 24TH ST,NaN,34.0266,-118.2676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317802,210911274,07/05/2021 12:00:00 AM,07/03/2021 12:00:00 AM,1900,9,Van Nuys,911,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,15600 VANOWEN ST,NaN,34.1939,-118.4732
317808,211204656,01/11/2021 12:00:00 AM,01/10/2021 12:00:00 AM,2300,12,77th Street,1259,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,400 E 74TH ST,NaN,33.9730,-118.2657
317846,210404776,01/21/2021 12:00:00 AM,01/19/2021 12:00:00 AM,1600,4,Hollenbeck,497,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,1400 S DE LA TORRE WY,NaN,34.0166,-118.2058
317849,211208872,03/19/2021 12:00:00 AM,03/19/2021 12:00:00 AM,1105,12,77th Street,1218,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,58TH ST,FIGUEROA ST,33.9897,-118.2827


In [17]:
dataset_510['Date Rptd'] = pd.to_datetime(dataset_510['Date Rptd'])
dataset_510['DATE OCC'] = pd.to_datetime(dataset_510['DATE OCC'])

dataset_510.sort_values(by="DATE OCC")


/var/folders/8t/cg46ygc52xq8ts0rysyjhfn40000gn/T/ipykernel_3242/4109199034.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset_510['Date Rptd'] = pd.to_datetime(dataset_510['Date Rptd'])
/var/folders/8t/cg46ygc52xq8ts0rysyjhfn40000gn/T/ipykernel_3242/4109199034.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_510['Date Rptd'] = pd.to_datetime(dataset_510['Date Rptd'])
/var/folders/8t/cg46ygc52xq8ts0rysyjhfn40000gn/T/ipykernel_3242/4109199034.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
15061,201604004,2020-01-01,2020-01-01,5,16,Foothill,1601,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,11900 COMETA AV,NaN,34.2866,-118.4125
9729,201104043,2020-01-02,2020-01-01,1830,11,Northeast,1145,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,2500 GLENDALE BL,NaN,34.1027,-118.2592
9728,201104038,2020-01-01,2020-01-01,800,11,Northeast,1149,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,100 S AVENUE 57,NaN,34.1074,-118.1978
1434,200204057,2020-01-02,2020-01-01,2000,2,Rampart,204,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,3400 PLATA ST,NaN,34.0777,-118.2821
1432,200204055,2020-01-02,2020-01-01,2200,2,Rampart,275,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,BURLINGTON,JAMES M WOOD,34.0519,-118.2755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292740,211011359,2021-08-16,2021-08-16,705,10,West Valley,1049,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,6300 BALBOA BL,NaN,0.0000,0.0000
317434,211815272,2021-08-16,2021-08-16,400,18,Southeast,1823,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,10100 S SAN PEDRO ST,NaN,33.9438,-118.2695
201876,210115282,2021-08-16,2021-08-16,1900,1,Central,138,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,BOYD ST,WALL ST,0.0000,0.0000
277674,211711552,2021-08-16,2021-08-16,510,17,Devonshire,1794,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,18500 PARTHENIA ST,NaN,34.2282,-118.5367


In [81]:
df1 = dataset_510['DATE OCC'].dt.date.value_counts().sort_index().reset_index()
df1.columns = ['DATE','Count']

df1['DATE'] = pd.to_datetime(df1['DATE']) - pd.to_timedelta(7, unit='d')
df1= df1.groupby([pd.Grouper(key='DATE', freq='W')])['Count'].sum()

df1 = pd.DataFrame(data=df1.values, columns=['Count'])
df1 = df1.values
df1
# df1 = dataset_510['DATE OCC'].dt.date.value_counts().sort_index().reset_index()
# df1.columns = ['DATE','Count']
# # df1.plot(x='DATE',y='Count')
# dataset_510.groupby([pd.Grouper(key='DATE OCC', freq='W')]).count()

array([[194],
       [344],
       [308],
       [317],
       [342],
       [312],
       [310],
       [306],
       [306],
       [322],
       [336],
       [307],
       [361],
       [412],
       [415],
       [412],
       [442],
       [394],
       [450],
       [413],
       [436],
       [400],
       [459],
       [468],
       [495],
       [514],
       [522],
       [431],
       [438],
       [438],
       [442],
       [414],
       [421],
       [382],
       [429],
       [384],
       [374],
       [388],
       [372],
       [387],
       [415],
       [382],
       [438],
       [411],
       [410],
       [409],
       [423],
       [397],
       [414],
       [403],
       [391],
       [385],
       [416],
       [400],
       [407],
       [442],
       [433],
       [484],
       [458],
       [466],
       [411],
       [435],
       [408],
       [391],
       [393],
       [408],
       [386],
       [398],
       [441],
       [460],
       [405],
      

In [92]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back),0]
        dataX.append(a)
        dataY.append(dataset[i + look_back,0])
    return np.array(dataX), np.array(dataY)

In [93]:
# dataset_510['DATE OCC'] = pd.to_datetime(dataset_510['DATE OCC']) - pd.to_timedelta(7, unit='d')

df1
train_size = int(len(df1) * 0.7)
print(train_size)
test_size = len(df1) - train_size
print(test_size)
train, test = df1[0:train_size,:], df1[train_size:len(df1),:]

# df1 = df1.drop(['DATE'], axis=1, index=None, columns=None, level=None, inplace=False, errors='raise')\

60
26


In [94]:
# I make my dataset
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [95]:
trainX.shape, trainY.shape, testX.shape, testY.shape

((59, 1), (59,), (25, 1), (25,))

In [96]:
trainX = np.reshape(trainX, (trainX.shape[0],1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0],1, testX.shape[1]))

In [97]:

from keras.models import Sequential
from keras.layers import Dense, LSTM



model = Sequential()
model.add(LSTM(8,return_sequences=True, input_shape=(1,look_back)))
model.add(LSTM(4,return_sequences=False, input_shape=(1,look_back)))

model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(trainX, trainY, epochs=500, batch_size=4, verbose=2)

2023-11-25 16:27:47.319072: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-25 16:27:56.751509: E tensorflow/core/lib/monitoring/collection_registry.cc:81] Cannot register 2 metrics with the same name: /tensorflow/api/enable_tensor_equality


AlreadyExistsError: Another metric with the same name already exists.